In [38]:
import re
import pandas as pd

def extract_substrings(text):
    # Находим фразы, начинающиеся и заканчивающиеся знаками доллара или начинающиеся и заканчивающиеся /begin{} и /end{}
    matches = re.findall(r'(\$.*?\$|\$\$.*?\$\$|/begin\{(.*?)\}(.*?)/end\{\2\})', text)
    return [match[0] if match[0] else f'/begin{{{match[1]}}}{match[2]}/end{{{match[1]}}}' for match in matches]
    # разделить поэтапно

def process_json(input_json):
    # Преобразуем JSON в DataFrame
    df = pd.DataFrame(input_json['data'])

    # Создаем новый столбец для хранения вычлененных подстрок
    df['new_column'] = df['column'].apply(extract_substrings)

    # Обновляем исходный столбец, заменяя подстроки на [[i]]
    for index, row in df.iterrows():
        for i, substring in enumerate(row['new_column']):
            df.at[index, 'column'] = df.at[index, 'column'].replace(substring, f'[[{i}]]')

    # Оставляем только необходимые столбцы
    result_df = df[['column', 'new_column']]
    return result_df

# Пример использования
json_data = {
    "data": [
        {"column": "Текст1 $важная_фраза1$ остальной текст"},
        {"column": "Текст2 /begin{важная_фраза2} остальной текст /end{важная_фраза2}"},
        {"column": "Текст3 $важная_фраза3$ $важная фраза 4$остальной текст"}
    ]
}

result_dataframe = process_json(json_data)
print(result_dataframe)


                              column  \
0       Текст1 [[0]] остальной текст   
1                       Текст2 [[0]]   
2  Текст3 [[0]] [[1]]остальной текст   

                                          new_column  
0                                  [$важная_фраза1$]  
1  [/begin{важная_фраза2} остальной текст /end{ва...  
2                [$важная_фраза3$, $важная фраза 4$]  


In [39]:
import re
import pandas as pd
import json

def extract_substrings(text):
    matches = re.findall(r'(\$.*?\$|/begin\{(.*?)\}(.*?)/end\{\2\})', text)
    return [match[0] if match[0] else f'/begin{{{match[1]}}}{match[2]}/end{{{match[1]}}}' for match in matches]

def process_json_file(file_path):
    # Загружаем данные из JSON-файла
    with open(file_path, 'r', encoding='utf-8') as json_file:
        input_json = json.load(json_file)

    # Извлекаем нужные данные из JSON
    data = {'column': input_json['body_markdown']}

    # Преобразуем данные в DataFrame, добавляя индекс
    df = pd.DataFrame(data)

    # Создаем новый столбец для хранения вычлененных подстрок
    df['new_column'] = df['column'].apply(extract_substrings)

    # Создаем новый DataFrame для хранения вычлененных подстрок
    extracted_phrases_df = pd.DataFrame({'extracted_phrases': [item for sublist in df['new_column'].tolist() for item in sublist]})

    # Заменяем подстроки на [[i]] в исходной таблице
    for index, row in df.iterrows():
        for i, substring in enumerate(row['new_column']):
            df.at[index, 'column'] = df.at[index, 'column'].replace(substring, f'[[{i}]]')

    # Возвращаем результат
    return df, extracted_phrases_df

# Пример использования
json_file_path = '/content/30_questions.json'
result_dataframe, extracted_phrases_dataframe = process_json_file(json_file_path)

# Выводим результаты
print("Исходная таблица:")
print(result_dataframe)
print("\nТаблица с вычлененными подстроками:")
print(extracted_phrases_dataframe)


Исходная таблица:
                                               column  \
0   [[0]] is a convex set and [[1]]  is the orthog...   
1   Let [[0]] be a simple complex Lie algebra with...   
10  Let *s*,*t*,*r* be non-zero complex numbers an...   
11  My question is about the meaning of linearity ...   
12  Let [[0]] be the open unit disk  and let [[1]]...   
13  The roots of [[0]] are simply [[1]]. They can ...   
14  I have two points A and B which lie in the int...   
15  [[0]]\r\n\r\nMy question is what happens when ...   
16  A paper I found while researching graver bases...   
17  We define [[0]] as the inverse limit [[1]]. On...   
18  Suppose I have the variational problem\r\n\r\n...   
19  I am trying to understand affine maps. In the ...   
2   Any ideas how to evaluate this integral over t...   
20  Suppose that you have a system:\r\n\begin{case...   
21  So instead of R, I now have U(10) which makes ...   
22  Note: I will be using capital letters to denot...   
23  &gt; The 